<a href="https://colab.research.google.com/github/praveenkumarbalakrishnan/tasks/blob/main/SIG731_Data_Wrangling_Task8HD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Title:** Task 8HD: SIG731-Data Wrangling

**Name:** Praveenkumar Balakrishnan

**Student Number:** 223029369

**Email Address:** s223029369@deakin.edu.au


# Import statements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import argparse
import re
import os
import time
import xml.etree.cElementTree as cetree

import pandas as pd

# Tasks

Selected Movies & TV dataset for this analysis

**Task 1**

Convert all the data tables (Badges, Comments, PostHistory, PostLinks, Posts, Tags, Users, Votes)
from XML to CSV, using custom code that you write yourself. Ideally, you should write a Python
function that takes a single input file name (.xml) and output file name (.csv) and performs the
conversion of a single dataset.

**Task 2**

Load the CSV files as pandas data frames


Column names for 8 dataset based on the schema documentation

In [153]:
file_path = '/content/drive/MyDrive/Inputs/'

badges_cols = ['Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased']
comments_cols = [
    'Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserDisplayName',
    'UserId', 'ContentLicense']
post_history_cols = [
    'Id', 'PostHistoryTypeId', 'PostId', 'RevisionGUID', 'CreationDate',
    'UserId', 'UserDisplayName', 'Comment', 'Text', 'ContentLicense']
post_links_cols = [
    'Id', 'CreationDate', 'PostId', 'RelatedPostId', 'LinkTypeId']
posts_cols = [
    'Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
    'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
    'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
    'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
    'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
    'ContentLicense']
tags_cols = [
    'Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId',
    'IsModeratorOnly', 'IsRequired']
users_cols = [
    'Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
    'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
    'ProfileImageUrl', 'AccountId']
votes_cols = [
    'Id', 'PostId', 'VoteTypeId', 'UserId',  'CreationDate', 'BountyAmount']

meta_dict = {
    'Badges': badges_cols, 'Comments': comments_cols,
    'PostHistory': post_history_cols, 'PostLinks': post_links_cols,
    'Posts': posts_cols, 'Tags': tags_cols, 'Users': users_cols,
    'Votes': votes_cols}

Parsed the xml elements, iterated in a loop and applied the regex functions to remove extra tags and other characters then load it in a csv file

In [146]:
def get_data(xml_file_name, cols_list):
    """
    This generator is to parse the xml data and yield as a row

    Parameters
    ----------
    file_name: str
      Input xml file name
    cols: list
      Column names of the input files
    
    Yield
    ------
    row: str
      Row from xml data
    """
    col_names = ''
    for col in cols_list:
      col_names = col_names + col + '|'
    col_names = col_names[:-1] + '\n'
    yield col_names

    for event, elem in cetree.iterparse(xml_file_name):
      row = ''
      if elem.tag == "row":
        for col in cols_list:
          if col in elem.attrib:
              val = elem.attrib[col].replace("|", " ").replace('\n', ' ')
              if col in ['Tags']:
                val = re.sub(r'<', '', val)
                val = re.sub(r'>', ' ', val)
              else:
                val = re.sub(r'<.?>', '', val)
                val = re.sub('[^A-Za-z0-9-:.]+', ' ', val)
                val = re.sub(' +', ' ', val)
              row = row + val.rstrip(' p') + '|'
          else:
              row = row + '|'
        row = row[:-1] + '\n'
        yield row
        elem.clear()

def convert_xml_to_csv(file_name, cols_list, path):
  """
  This function is to convert xml to csv and write as csv file
  and upper value

  Parameters
  ----------
  file_name: str
    Input filename without the extension like ".xml"
  cols: list
    Column names of the input files
  
  Return
  ------
  csv_filename: str
    Output csv filename
  """
  xml_file_name = os.path.join(path, file_name + '.xml')
  output_file_name = os.path.join(path, file_name + '.csv')

  data = get_data(xml_file_name, cols_list)
  with open(output_file_name, 'w') as f:
      for item in data:
          f.write(item)
  return output_file_name

Invoked xml conversion function and got the filename then used pandas read_csv to load the data into separate dataframes.
Note: Used dictionary to store the dataframe in the loop

In [154]:
data_dict = {}
for k, v in meta_dict.items():
  csv_file_name = convert_xml_to_csv(k, v, file_path)
  data_dict[k] = pd.read_csv(csv_file_name, delimiter='|')

Loaded the dataframe to join in the upcoming step

In [156]:
badges_df = data_dict['Badges']
comments_df = data_dict['Comments']
post_history_df = data_dict['PostHistory']
post_links_df = data_dict['PostLinks']
posts_df = data_dict['Posts']
tags_df = data_dict['Tags']
users_df = data_dict['Users']
votes_df = data_dict['Votes']